# FC Metz - Exploration des APIs

Ce notebook permet d'explorer les données disponibles via les APIs StatsBomb et SkillCorner.
怎么探索和分析数据

## 1. API StatsBomb

In [3]:
from statsbombpy import sb
import sys, os

#verifier si on est dans le bon dossier
_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "API" else os.getcwd()
sys.path.insert(0, _root)

#from config.py to import the credentials
from config import STATSBOMB_CREDS
creds = STATSBOMB_CREDS

In [4]:
# Voir toutes les compétitions disponibles
#拉取所有可用的赛事，comps是一个dataframe,每一行是一个【赛事，赛季】
comps = sb.competitions(creds=creds)
print(f"Nombre de compétitions-saisons: {len(comps)}")
comps.head(20)

Nombre de compétitions-saisons: 7


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,75,316,Sweden,Allsvenskan,male,False,False,2026,2026-01-26T05:45:29.796726,None,None,None
1,75,315,Sweden,Allsvenskan,male,False,False,2025,2026-02-09T07:54:09.000000,2026-02-09T07:54:09.000000,2026-02-09T07:54:09.000000,2026-02-09T07:54:09.000000
2,88,316,Norway,Eliteserien,male,False,False,2026,2025-12-22T12:08:17.011976,None,None,None
3,88,315,Norway,Eliteserien,male,False,False,2025,2026-02-13T18:56:36.000000,2026-02-13T18:56:36.000000,2026-02-13T18:56:36.000000,2026-02-13T18:56:36.000000
4,7,318,France,Ligue 1,male,False,False,2025/2026,2026-02-23T21:32:52.000000,2026-02-23T21:32:52.000000,2026-02-23T21:32:52.000000,2026-02-23T21:32:52.000000
5,7,317,France,Ligue 1,male,False,False,2024/2025,2026-01-22T03:20:39.000000,2026-01-22T03:20:39.000000,2026-01-22T03:20:39.000000,2026-01-22T03:20:39.000000
6,8,318,France,Ligue 2,male,False,False,2025/2026,2026-02-23T20:57:22.000000,2026-02-23T20:57:22.000000,2026-02-23T20:57:22.000000,2026-02-23T20:57:22.000000


In [5]:
# Filtrer pour la Ligue 1
ligue1 = comps[comps['competition_name'].str.contains('Ligue 1', case=False, na=False)]
print("Ligue 1 saisons disponibles:")
ligue1[['competition_id', 'season_id', 'competition_name', 'season_name', 'country_name']]

Ligue 1 saisons disponibles:


,competition_id,season_id,competition_name,season_name,country_name
4,7,318,Ligue 1,2025/2026,France
5,7,317,Ligue 1,2024/2025,France


In [6]:
# Charger les matchs de la saison la plus récente
latest = ligue1.sort_values('season_id', ascending=False).iloc[0]
comp_id = int(latest['competition_id'])
season_id = int(latest['season_id'])
print(f"Compétition: {latest['competition_name']} - {latest['season_name']}")
print(f"competition_id={comp_id}, season_id={season_id}")

matches = sb.matches(competition_id=comp_id, season_id=season_id, creds=creds)
print(f"\nNombre de matchs: {len(matches)}")
print(f"  - available (terminés, avec events): {(matches['match_status'] == 'available').sum()}")
print(f"  - scheduled (à venir): {(matches['match_status'] == 'scheduled').sum()}")

# Les 5 prochains matchs (à venir), triés par date
from IPython.display import display
prochains = matches[matches['match_status'] == 'scheduled'].sort_values('match_date').head(5)
print("--- Les 5 prochains matchs ---")
display(prochains[['match_date', 'kick_off', 'home_team', 'away_team', 'match_status']])

# Les 5 matchs les plus récents terminés
termines = matches[matches['match_status'] == 'available'].sort_values('match_date', ascending=False).head(5)
print("--- Les 5 matchs les plus récents terminés ---")
display(termines[['match_date', 'kick_off', 'home_team', 'away_team', 'home_score', 'away_score']])

Compétition: Ligue 1 - 2025/2026
competition_id=7, season_id=318

Nombre de matchs: 306
  - available (terminés, avec events): 207
  - scheduled (à venir): 99
--- Les 5 prochains matchs ---


,match_date,kick_off,home_team,away_team,match_status
21,2026-02-27,19:45:00.000,Strasbourg,Lens,scheduled
223,2026-02-28,20:05:00.000,Le Havre,Paris Saint-Germain,scheduled
10,2026-02-28,18:00:00.000,AS Monaco,Angers,scheduled
26,2026-02-28,16:00:00.000,Rennes,Toulouse,scheduled
158,2026-03-01,16:15:00.000,Metz,Stade Brestois,scheduled


--- Les 5 matchs les plus récents terminés ---


,match_date,kick_off,home_team,away_team,home_score,away_score
152,2026-02-22,16:15:00.000,OGC Nice,Lorient,3.0,3.0
25,2026-02-22,16:15:00.000,Angers,Lille,0.0,1.0
259,2026-02-22,19:45:00.000,Strasbourg,Lyon,3.0,1.0
279,2026-02-22,14:00:00.000,Auxerre,Rennes,0.0,3.0
207,2026-02-22,16:15:00.000,Nantes,Le Havre,2.0,0.0


In [7]:
# Charger les événements du match le plus récent (passes, tirs, dribbles, etc.)
#单场比赛事件
matches_disponibles = matches[matches['match_status'] == 'available']
if len(matches_disponibles) == 0:
    print("Aucun match avec données events disponible.")
else:
    from IPython.display import display
    # Trier par date décroissante, prendre le plus récent
    match_row = matches_disponibles.sort_values('match_date', ascending=False).iloc[0]
    match_id = int(match_row['match_id'])
    print(f"Match: {match_row['home_team']} vs {match_row['away_team']}")
    print(f"Score: {match_row['home_score']}-{match_row['away_score']}\n")

    events = sb.events(match_id=match_id, creds=creds)
    print(f"Nombre d'événements: {len(events)}")
    print(f"Types d'événements: {events['type'].unique().tolist()[:15]}...\n")
    
    # Les premiers événements (Starting XI, Half Start) n'ont pas player/location -> NaN
    # Filtrer les événements avec joueur et afficher colonnes clés
    events_avec_joueur = events[events['player'].notna()]
    cols_cles = ['type', 'player', 'team', 'minute', 'second', 'location']
    if 'pass_end_location' in events.columns:
        cols_cles.append('pass_end_location')
    print("--- Aperçu des 10 premiers événements (Pass, Shot, etc.) ---")
    display(events_avec_joueur[cols_cles].head(10))
   
    """前几行是结构性事件
    events.head(10) 的前几行多为 Starting XI、Half Start 等非比赛行为事件，没有 player、location 等字段，所以会显示为 NaN。
    列太多且大多为空;   events 有 100+ 列，很多是特定事件类型才有的（如 shot_statsbomb_xg 只对射门有值），对大部分行是 NaN，显示会很乱。"""


Match: OGC Nice vs Lorient
Score: 3.0-3.0

Nombre d'événements: 3562
Types d'événements: ['Starting XI', 'Half Start', 'Pass', 'Ball Receipt*', 'Carry', 'Pressure', 'Miscontrol', 'Duel', 'Block', 'Dispossessed', 'Clearance', 'Interception', 'Ball Recovery', 'Goal Keeper', 'Shot']...

--- Aperçu des 10 premiers événements (Pass, Shot, etc.) ---


/Users/hazelchan00324/Library/Python/3.9/lib/python/site-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)


,type,player,team,minute,second,location,pass_end_location
6,Pass,Morgan Sanson,OGC Nice,0,0,"[60.0, 40.0]","[36.0, 39.0]"
7,Pass,Dante Bonfim da Costa Santos,OGC Nice,0,5,"[41.8, 39.9]","[120.0, 74.2]"
8,Pass,Montassar Omar Talbi,Lorient,0,33,"[6.5, 48.4]","[3.4, 38.7]"
9,Pass,Yvon Landry Mvogo,Lorient,0,37,"[3.6, 39.9]","[22.1, 59.4]"
10,Pass,Montassar Omar Talbi,Lorient,0,41,"[37.3, 57.2]","[91.6, 43.4]"
11,Pass,Kojo Peprah Oppong,OGC Nice,0,45,"[27.7, 37.0]","[42.7, 27.4]"
12,Pass,Noah Cadiou,Lorient,0,49,"[77.4, 52.7]","[85.6, 57.1]"
13,Pass,Ali Abdi,OGC Nice,0,50,"[34.5, 23.0]","[42.2, 38.4]"
14,Pass,Laurent Abergel,Lorient,0,52,"[77.9, 41.7]","[85.3, 42.9]"
15,Pass,Pablo Pagis,Lorient,0,54,"[83.6, 42.4]","[78.6, 49.6]"


In [8]:
# Stats de saison par joueur 球员赛季统计
try:
    from IPython.display import display
    player_stats = sb.player_season_stats(competition_id=comp_id, season_id=season_id, creds=creds)
    print(f"Nombre de joueurs avec stats: {len(player_stats)}")
    print(f"Colonnes disponibles: {player_stats.columns.tolist()[:20]}...")
    display(player_stats[['player_name', 'team_name', 'player_season_minutes', 
                   'player_season_goals_90', 'player_season_np_xg_90']].head(10))
except Exception as e:
    print(f"Erreur: {e}")

Nombre de joueurs avec stats: 527
Colonnes disponibles: ['account_id', 'player_id', 'player_name', 'team_id', 'team_name', 'competition_id', 'competition_name', 'season_id', 'season_name', 'country_id', 'birth_date', 'player_female', 'player_first_name', 'player_last_name', 'player_known_name', 'player_weight', 'player_height', 'player_season_minutes', 'player_season_np_xg_per_shot', 'player_season_np_xg_90']...


,player_name,team_name,player_season_minutes,player_season_goals_90,player_season_np_xg_90
0,Andrew Omobamidele,Strasbourg,779.80005,0.000000,0.037294
1,Hyeok-Kyu Kwon,Nantes,823.23340,0.000000,0.013067
2,Hyeon-seok Hong,Nantes,311.25000,0.000000,0.103786
3,Kang-In Lee,Paris Saint-Germain,1106.06670,0.162739,0.141059
4,Matvey Safonov,Paris Saint-Germain,590.16660,0.000000,0.000000
5,Aleksandr Golovin,AS Monaco,1200.31650,0.149960,0.114709
6,Mbwana Ally Samatta,Le Havre,994.10000,0.000000,0.180185
7,Folarin Balogun,AS Monaco,1467.38330,0.306668,0.373019
8,Tanner Tessmann,Lyon,1726.00010,0.052144,0.082498
9,Mark McKenzie,Toulouse,1819.41670,0.000000,0.016349


## 2. API SkillCorner

### Présentation

**SkillCorner** fournit des **données physiques/tracking** : distance parcourue, sprints, accélérations, vitesse max, etc. Complémentaire à StatsBomb (événements), utilisé pour analyser la charge physique des joueurs.

**Documentation** : [https://skillcorner.com/api/docs/](https://skillcorner.com/api/docs/)

| Endpoint | Description | Utilisation notebook |
|----------|-------------|----------------------|
| `get_seasons` | Liste des saisons | Filtrage saison la plus récente |
| `get_competitions` | Ligues par saison | Filtrage France / Ligue 1 |
| `get_competition_editions` | Éditions (ex: FRA Ligue 1 2025/26) | Sélection édition FC Metz |
| `get_teams` | Équipes par édition | Liste équipes Ligue 1 |
| `get_physical` | Données physiques (joueur × match) | Métriques clés : distance, sprints |

为什么只选择这几个分析，和fc metz，saison赛季，competition联赛，edition fra-figue1版本，teams优先fc

In [9]:
from skillcorner.client import SkillcornerClient
import sys, os
_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
sys.path.insert(0, _root)

# from config.py to import the credentials
from config import SKILLCORNER_USERNAME, SKILLCORNER_PASSWORD
client = SkillcornerClient(username=SKILLCORNER_USERNAME, password=SKILLCORNER_PASSWORD)

### 2.1 Saisons
Liste des saisons disponibles. On conserve les **5 plus récentes** pour le projet FC Metz.

In [10]:
# 2.1 Saisons disponibles
import pandas as pd
from IPython.display import display
seasons = client.get_seasons()
df_seasons = pd.DataFrame(seasons)[['id', 'name', 'start_year', 'end_year']]
df_seasons = df_seasons.sort_values('id', ascending=False).head(5)
print(f"Nombre total de saisons SkillCorner : {len(seasons)}")
print("→ 5 saisons les plus récentes :")
display(df_seasons)

Nombre total de saisons SkillCorner : 33
→ 5 saisons les plus récentes :


,id,name,start_year,end_year
0,130,2026,2026,2026
1,129,2025/2026,2025,2026
2,128,2025,2025,2025
3,95,2024/2025,2024,2025
27,62,2010,2010,2010


### 2.2 Compétitions (filtrées France)
Ligues disponibles pour la saison la plus récente. **Filtre** : France (FRA) ou contenant "Ligue".

In [11]:
# 2.2 Compétitions (saison récente, filtre France)
latest_season_id = max(s['id'] for s in seasons)
competitions = client.get_competitions(params={'season': latest_season_id})
df_comp = pd.DataFrame([{'id': c.get('id'), 'name': c.get('name', 'N/A'), 
    'area': c.get('area', {}).get('name', '') if isinstance(c.get('area'), dict) else ''} for c in competitions])
mask_fra = df_comp['name'].str.contains('Ligue|France|L1', case=False, na=False) | df_comp['area'].str.contains('France', na=False)
df_comp_fra = df_comp[mask_fra].head(10)
print(f"Compétitions saison {latest_season_id} : {len(competitions)} total, {mask_fra.sum()} France/Ligue")
display(df_comp_fra[['id', 'name', 'area']])

Compétitions saison 130 : 42 total, 0 France/Ligue


,id,name,area


### 2.3 Éditions (FRA Ligue 1 uniquement)
Une édition = une ligue pour une saison donnée (ex: FRA - Ligue 1 - 2025/2026). On filtre **FRA** pour le projet FC Metz.

In [12]:
# 2.3 Éditions FRA Ligue 1
editions = client.get_competition_editions()
ligue1_all = [e for e in editions if 'ligue 1' in str(e.get('name', '')).lower() 
             or 'ligue 1' in str(e.get('competition', {}).get('name', '')).lower()]
ligue1_fra = [e for e in ligue1_all if 'FRA' in str(e.get('name', '')).upper() and 'playoff' not in str(e.get('name', '')).lower()]
df_ed = pd.DataFrame([{'id': e.get('id'), 'name': e.get('name'), 
    'comp': e.get('competition', {}).get('name'), 'season': e.get('season', {}).get('name')} 
    for e in (ligue1_fra if ligue1_fra else ligue1_all)])
df_ed = df_ed.sort_values('id', ascending=False).head(5)
ligue1_editions = ligue1_fra if ligue1_fra else ligue1_all  # pour les cellules suivantes
print(f"Éditions Ligue 1 : {len(ligue1_all)} total, {len(ligue1_fra)} FRA (hors playoffs)")
display(df_ed)

Éditions Ligue 1 : 17 total, 10 FRA (hors playoffs)


,id,name,comp,season
9,1232,FRA - Ligue 1 - 2025/2026,Ligue 1,2025/2026
8,906,FRA - Ligue 1 - 2024/2025,Ligue 1,2024/2025
7,548,FRA - Ligue 1 - 2023/2024,Ligue 1,2023/2024
6,392,FRA - Ligue 1 - 2022/2023,Ligue 1,2022/2023
5,242,FRA - Ligue 1 - 2021/2022,Ligue 1,2021/2022


### 2.4 Équipes et données physiques
| Donnée | Description |
|--------|-------------|
| **Équipes** | Liste des équipes pour l'édition sélectionnée (priorité FC Metz) |
| **Données physiques** | Distance, sprints, haute intensité — par joueur × match |

In [13]:
# 2.4 Équipes et données physiques (FC Metz prioritaire)
if ligue1_editions:
    ed_id = ligue1_editions[-1]['id']  # édition la plus récente
    teams = client.get_teams(params={'competition_edition': ed_id})
    df_teams = pd.DataFrame([{'id': t.get('id'), 'name': t.get('name')} for t in teams])
    # Priorité FC Metz : le mettre en premier si présent
    mask_metz = df_teams['name'].str.contains('Metz', case=False, na=False)
    if mask_metz.any():
        df_teams = pd.concat([df_teams[mask_metz], df_teams[~mask_metz]]).reset_index(drop=True)
    print(f"Équipes pour l'édition {ed_id} : {len(teams)}")
    display(df_teams.head(10))
    
    if teams:
        metz_team = next((t for t in teams if 'metz' in str(t.get('name', '')).lower()), None)
        team = metz_team or teams[0]
        team_id = team['id']
        team_name = team.get('name', '')
        physical = client.get_physical(params={'team': team_id})
        print(f"\nDonnées physiques — {team_name} : {len(physical)} enregistrements")
        if physical:
            df_phys = pd.DataFrame(physical)
            cols = ['player_name', 'match_date', 'total_distance_full_all', 'sprint_count_full_all', 
                    'hsr_distance_full_all', 'minutes_full_all', 'total_metersperminute_full_all']
            cols = [c for c in cols if c in df_phys.columns]
            df_phys = df_phys.sort_values('match_date', ascending=False)
            display(df_phys[cols].head(10))

Équipes pour l'édition 1232 : 18


,id,name
0,97,FC Metz
1,85,RC Lens
2,65,AJ Auxerre
3,66,FC Nantes
4,69,Le Havre AC
5,70,OGC Nice
6,72,FC Lorient
7,74,Toulouse FC
8,78,LOSC Lille
9,98,Angers SCO



Données physiques — FC Metz : 334 enregistrements


,player_name,match_date,total_distance_full_all,sprint_count_full_all,hsr_distance_full_all,minutes_full_all,total_metersperminute_full_all
333,Nathan Mbala,2026-02-21,793.0,1.0,23.0,5.23,151.53
203,Koffi Franck Kouao,2026-02-21,7408.0,16.0,461.0,69.82,106.11
60,Fodé Ballo-Touré,2026-02-21,7649.0,11.0,511.0,69.82,109.56
82,Jessy Deminguet,2026-02-21,3789.0,2.0,220.0,27.28,138.88
105,Georgiy Tsitaishvili,2026-02-21,10846.0,18.0,692.0,91.87,118.06
108,Giorgi Kvilitaia,2026-02-21,1820.0,1.0,134.0,15.48,117.55
124,Habib Mouhamadou Diallo,2026-02-21,10438.0,10.0,495.0,97.10,107.50
34,Jean-Philippe Gbamin,2026-02-21,10577.0,5.0,609.0,97.10,108.93
183,Boubacar Traoré,2026-02-21,11772.0,7.0,640.0,97.10,121.24
14,Bouna Sarr,2026-02-21,3356.0,9.0,300.0,27.28,123.01


## 3. Lancer le Pipeline Complet

Pour lancer le pipeline complet depuis le terminal :
```bash
python main.py --quick  # Mode rapide pour tester
python main.py          # Pipeline complet
```

In [14]:
# Explorer la base de données après le pipeline (PostgreSQL)
import pandas as pd
import sys
import os

_project_root = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == "notebooks" else os.getcwd()
sys.path.insert(0, _project_root)
from config import POSTGRES_CONFIG, DB_SCHEMA

try:
    conn = __import__("psycopg2").connect(**POSTGRES_CONFIG)
    conn.autocommit = True  # pour pandas read_sql
    
    # Voir les tables du schéma fc_metz
    tables = pd.read_sql(
        f"""SELECT tablename FROM pg_tables WHERE schemaname = '{DB_SCHEMA}'""",
        conn
    )
    print("Tables dans la base:")
    for t in tables['tablename']:
        count = pd.read_sql(f"SELECT COUNT(*) as n FROM {DB_SCHEMA}.{t}", conn)['n'][0]
        print(f"  {t}: {count} lignes")
    
    # Aperçu des joueurs
    players = pd.read_sql(f"""
        SELECT player_name, statsbomb_player_id, skillcorner_player_id, 
               transfermarkt_player_id, market_value, nationality, primary_position
        FROM {DB_SCHEMA}.players 
        WHERE statsbomb_player_id IS NOT NULL
        LIMIT 20
    """, conn)
    print("\nAperçu des joueurs:")
    display(players)
    
    conn.close()
except Exception as e:
    print(f"Base non trouvée. Lancez d'abord: python main.py --quick")
    print(f"Erreur: {e}")

Tables dans la base:
  competitions: 4 lignes
  seasons: 4 lignes
  matches: 306 lignes
  teams: 18 lignes
  match_lineups: 8258 lignes
  players: 1822 lignes
  player_id_mapping: 696 lignes
  events: 739710 lignes
  player_season_stats: 2623 lignes
  player_match_physical: 11541 lignes
  player_fused: 1820 lignes

Aperçu des joueurs:


/var/folders/jj/3jmj_tmx0s3bbfd8q6j93t7h0000gn/T/ipykernel_15937/3506288015.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql(
/var/folders/jj/3jmj_tmx0s3bbfd8q6j93t7h0000gn/T/ipykernel_15937/3506288015.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  count = pd.read_sql(f"SELECT COUNT(*) as n FROM {DB_SCHEMA}.{t}", conn)['n'][0]
/var/folders/jj/3jmj_tmx0s3bbfd8q6j93t7h0000gn/T/ipykernel_15937/3506288015.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  players = pd.read_sql(f"""


,player_name,statsbomb_player_id,skillcorner_player_id,transfermarkt_player_id,market_value,nationality,primary_position
0,Lucas Chevalier,39600,34467.0,463600,"35,00 mio. €",France,Lucas ChevalierGardien de but
1,Lukáš Hrádecký,8667,6358.0,48015,"1,80 mio. €",Finland,Lukas HradeckyGardien de but
2,Nuno Mendes,41092,13177.0,616341,"75,00 mio. €",Portugal,Nuno MendesArrière gauche
3,Lucas Hernández Pi,5484,875.0,281963,"20,00 mio. €","France, Espagne",Lucas HernándezArrière gauche
4,Sael Kumbedi Nseke,181010,100114.0,None,None,France,None
5,Younes Namli,8434,11036.0,None,None,Denmark,None
6,João Neves,143859,154093.0,670681,"110,00 mio. €",Portugal,João NevesMilieu central
7,Hamza Koutoune,416300,799254.0,None,None,Morocco,None
8,Kang-In Lee,22740,20236.0,557149,"25,00 mio. €",Corée du Sud,Kang-in LeeMilieu offensif
9,Morgan Guilavogui,47351,33150.0,None,None,Guinea,None
